In [ ]:
%%capture
!pip install -U bitsandbytes

import os
os.kill(os.getpid(), 9) #restart the session

In [1]:
%%capture
!pip install alfworld[full]
!alfworld-download
!pip install colorama

In [2]:
import yaml
import alfworld
import alfworld.agents.environment.alfred_tw_env

In [3]:
from IPython.display import HTML, display
import uuid
import json
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')
from huggingface_hub import login
login(HF_TOKEN)

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enables 4-bit quantization
    bnb_4bit_quant_type="nf4",  # NormalFloat4 (NF4) for better precision
    bnb_4bit_compute_dtype=torch.float16,  # Compute in float16 (reduce memory usage)
    bnb_4bit_use_double_quant=True  # Double quantization for additional compression
)


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)


model.to(DEVICE)

print(f"✅ Model loaded successfully on {DEVICE} with 4-bit quantization!")

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

✅ Model loaded successfully on cuda with 4-bit quantization!


In [7]:
from transformers import pipeline
# generator = pipeline('text-generation', model = model, tokenizer=tokenizer)
tokenizer.eos_token = "\n"
tokenizer.pad_token_id = tokenizer.eos_token_id  # Prevents warning
model.config.pad_token_id = tokenizer.eos_token_id
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer,eos_token_id=tokenizer.eos_token_id,  # Stop generation at end-of-sequence token
        pad_token_id=tokenizer.pad_token_id,
                        max_new_tokens = 200, device_map="auto")

Device set to use cuda:0


In [8]:
system_prompt = f'''Interact with a household to solve a task. Imagine you are an intelligent agent in a household environment and your target is to perform actions to complete the task goal.
At the beginning of your interactions, you will be given the detailed description of the current environment and your goal to accomplish.

For each of your turn, you will be given a list of actions which you can choose one to perform in this turn.

You can choose either of given option: \"THOUGHT\" or \"ACTION\". If you choose \"THOUGHT\", you should first think about the current condition and plan for your future actions,
and then output your action in this turn.
Your output must strictly follow this format:\"THOUGHT: your thoughts.\nACTION: your next action\n\";

If you choose \"ACTION\", you should directly output the action in this turn. Your output must strictly follow this format:\"ACTION: your next action\n\".

 After your each turn, the environment will give you immediate feedback based on which you plan your next few steps.
 if the envrionment output \"Nothing happened\", that means the previous action is invalid and you should try more options.
 \n Reminder: \n1. the action must be chosen from the given available actions. Any actions except provided available actions will be regarded as **illegal**. \n2.Think when necessary, try to act directly more in the process.
'''

In [9]:
with open('base_config.yaml') as reader:
    config = yaml.safe_load(reader)

split = "eval_out_of_distribution"
# config['env']['type'] = 'AlfredTwEnv'
# print(config['env']['type'])
env = getattr(alfworld.agents.environment.alfred_tw_env, 'AlfredTWEnv')(config, train_eval=split)
env = env.init_env(batch_size=1)

Initializing AlfredTWEnv...


100%|██████████| 341/341 [00:01<00:00, 337.90it/s]

Overall we have 134 games in split=eval_out_of_distribution
Evaluating with 134 games


In [24]:
from typing import List, Dict


class EnvironmentHistory:
    def __init__(self, base_query: str, start_info, memory: List[str], history: List[Dict[str, str]] = []) -> None:
        self._cur_query: str = f'{_get_base_query(base_query, start_info, memory)}'
        self._history: List[Dict[str, str]] = history
        self._last_action: str = ''
        self._is_exhausted: bool = False

    def add(self, label: str, value: str) -> None:
        assert label in ['action', 'observation', 'human_edit']
        self._history += [{
            'label': label,
            'value': value,
        }]
        if label == 'action':
            if value == self._last_action:
                self._is_exhausted = True
            else:
                self._last_action = value

    def check_is_exhausted(self) -> bool:
        return self._is_exhausted

    def reset(self) -> None:
        self._history = []

    def __str__(self) -> str:
        s: str = self._cur_query + '\n'
        for i, item in enumerate(self._history):
            if item['label'] == 'action':
                s += f'> {item["value"]}'
            elif item['label'] == 'observation':
                s += item['value']
            # NOT CURRENTLY SUPPORTED
            elif item['label'] == 'human_edit':
                s += f'[human edit]: {item["value"]}'
            if i != len(self._history) - 1:
                s += '\n'
        return s

def _get_base_query(base_query: str, start_info: str, memory: List[str]) -> str:
    query = base_query

    # add memory if it exists
    if len(memory) > 0:
        query += '\n\nYour memory for the task below:'
        for i, m in enumerate(memory):
            query += f'\nTrial {i}:\n{m.strip()}'
    query += f"\nHere is the task:\n{start_info}"
    return query


In [63]:
from typing import List, Dict, Any


with open("reflexion_few_shot_examples.txt", 'r') as f:
    FEW_SHOT_EXAMPLES = f.read()

def _get_scenario(s: str) -> str:
    """Parses the relevant scenario from the experience log."""
    return s.split("Here is the task:")[-1].strip()

def _generate_reflection_query(log_str: str, memory: List[str]) -> str:
    """Allows the Agent to reflect upon a past experience."""
    scenario: str = _get_scenario(log_str)
    query: str = f"""You will be given the history of a past experience in which you were placed in an environment and given a task to complete.
                      You were unsuccessful in completing the task. Do not summarize your environment, but rather think about the strategy and path
                      you took to attempt to complete the task. Devise a concise, new plan of action that accounts for your mistake with reference
                      to specific actions that you should have taken. For example, if you tried A and B but forgot C,
                      then devise a plan to achieve C with environment-specific actions. You will need this later when you are solving the same task.
                      Give your plan after "Plan". Here are two examples:

{FEW_SHOT_EXAMPLES}

{scenario}"""

    if len(memory) > 0:
        query += '\n\nPlans from past attempts:\n'
        for i, m in enumerate(memory):
            query += f'Trial #{i}: {m}\n'

    query += '\n\nNew plan:'
    return query

def update_memory(trial_log_path: str, env_configs: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """Updates the given env_config with the appropriate reflections."""
    with open(trial_log_path, 'r') as f:
        full_log: str = f.read()

    env_logs: List[str] = full_log.split('#####\n\n#####')
    assert len(env_logs) == len(env_configs), print(f'bad: {len(env_logs)}, {len(env_configs)}')
    for i, env in enumerate(env_configs):
        # if unsolved, get reflection and update env config
        if not env['is_success'] and not env['skip']:
            if len(env['memory']) > 3:
                memory: List[str] = env['memory'][-3:]
            else:
                memory: List[str] = env['memory']
            reflection_query: str = _generate_reflection_query(env_logs[i], memory)
            reflection: str = llm_pipeline(reflection_query)[0]['generated_text'] # type: ignore
            env_configs[i]['memory'] += [reflection]

    return env_configs

In [66]:
trial_log_path = "trial_0.log"
i=0
env_configs:List[Dict[str, Any]] = []
env_configs += [{
                'name': f'env_{i}',
                'memory': [],
                'is_success': False,
                'skip': False
            }]

In [67]:


def process_ob(ob):
    if ob.startswith('You arrive at loc '):
        ob = ob[ob.find('. ')+2:]
    return ob

def alfworld_run(memory, conversation_history, to_print=False, ob=''):
    if len(memory)>3:
      env_history = EnvironmentHistory(base_query=system_prompt, start_info=ob, memory=memory[-3:])
    else:
      env_history = EnvironmentHistory(base_query=system_prompt, start_info=ob, memory=memory)

    env_history.reset()

    prompt = ''

    if to_print:
        sys.stdout.flush()

    for i in range(1, 2):
        with torch.no_grad():
            llm_response = llm_pipeline(conversation_history, pad_token_id=llm_pipeline.tokenizer.eos_token_id)[0]['generated_text']
        llm_response = llm_response[-1]['content']

        action =  llm_response.split("ACTION:")[-1].strip() if "ACTION:" in llm_response else llm_response.strip()
        print(f'i = {i}, action = {action}')
        env_history.add("action", action)

        # print(f'llm_response = {llm_response[-1]}')
        observation, reward, done, info = env.step([action])
        observation, reward, done = process_ob(observation[0]), info['won'][0], done[0]
        env_history.add("observation", observation)

        if action.startswith('THOUGHT:'):
            observation = 'OK.'
        if to_print:
            print(f'Act {i}: {llm_response}\nObs {i}: {observation}, Reward = {reward}')
            print('.'*10)
            sys.stdout.flush()

        conversation_history.append({'role':'assistant', 'content':llm_response, 'reward':reward})
        admissible_commands = ", ".join(info['admissible_commands'][0])
        conversation_history.append({'role':'user', 'content':observation + "\n Here is the updated list of the available actions provided to you: " +admissible_commands, 'reward':None})

        if done:
            return reward, conversation_history, env_history, True

        elif env_history.check_is_exhausted():
            return reward, conversation_history, env_history, False

    return 0, conversation_history, env_history, False







# with open("alfworld_conversations.json", "r", encoding="utf-8") as json_file:
#     loaded_data = json.load(json_file)

# print("Loaded Conversation History:", json.dumps(loaded_data, indent=2, ensure_ascii=False))

In [68]:
prefixes = {
    'pick_and_place': 'put',
    'pick_clean_then_place': 'clean',
    'pick_heat_then_place': 'heat',
    'pick_cool_then_place': 'cool',
    'look_at_obj': 'examine',
    'pick_two_obj': 'puttwo'
}

cnts = [0] * 6
rs = [0] * 6

conversations_traj = []

# first 0 to 9
# second 10 to 19
num_successes = 0
num_additional_successes = 0
num_envs = 1
env_config = env_configs[0]
for _ in range(28,29):
    ob, info = env.reset()
    ob = '\n'.join(ob[0].split('\n\n')[1:])
    name = '/'.join(info['extra.gamefile'][0].split('/')[-3:-1])
    print(f"using {name}")
    if env_config["is_success"]:
            num_successes += 1

            # log to world log
            # with open(world_log_path, 'a') as wf:
            #     wf.write(f'Environment #{_} Trial #{trial_idx}: SUCCESS\n')
            with open(trial_log_path, 'a') as wf:
                wf.write(f'\n#####\n\nEnvironment #{_}: Success\n\n#####\n')
            continue

    alfworld_id = str(uuid.uuid4())

    conversation_history = []
    conversation_history.append({"role":"user", "content":system_prompt})
    response = llm_pipeline(conversation_history, pad_token_id=llm_pipeline.tokenizer.eos_token_id)  #calling pipleline
    assistant_response = response[0]['generated_text']
    conversation_history.append({'role':"assistant", 'content': assistant_response[-1]['content']})
    for i, (k, v) in enumerate(prefixes.items()):
        if name.startswith(k):
            # prompt = 'Interact with a household to solve a task. Here are two examples.\n' + d[f'react_{v}_1'] + d[f'react_{v}_0'] + '\nHere is the task.\n'
            # prompt = conversation_history.append({'role':'user', 'content':'Here is your task.'+ob})
            # print('admissible_commands', info['admissible_commands'])
            admissible_commands = ", ".join(info['admissible_commands'][0])
            conversation_history.append({'role':'user', 'content':'Here is your task.'+ob + "\n Given are the the available actions provided to you: " +admissible_commands})
            print(k, v)
            # print(print(f'Observation = {ob}'))
            r, conversation_history, final_env_history, is_success = alfworld_run(env_config["memory"],conversation_history, ob=ob)
            if is_success:
              status_str = f'Environment #{_} Trial #{0}: SUCCESS'
              num_additional_successes += 1
            else:
              status_str = f'Environment #{_} Trial #{0}: FAIL'

            with open(trial_log_path, 'a') as wf:
                    wf.write(f'\n#####\n\nEnvironment #{_}:\n{str(final_env_history)}\n\nSTATUS: {"OK" if is_success else "FAIL"}\n\n#####\n')
            # print("env_history =", env_history)
            rs[i] += r
            cnts[i] += 1
    # print(_+1, 'r', r, 'rs', rs, 'cnts', cnts, 'sum(rs)/sum(cnts)', sum(rs) / sum(cnts))
    conversations_traj.append({
        'traj_id': alfworld_id,
        'conversations': conversation_history
    })
    print('------------------------------------------------\n')

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


using pick_clean_then_place_in_recep-Knife-None-CounterTop-10/trial_T20190909_110445_675754


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


pick_clean_then_place clean
i = 1, action = go to cabinet 1
------------------------------------------------



In [55]:
with open('trial_0.log',"r") as log_file:
  logs = log_file.read()

print("content =", logs)

content = 
#####

Environment #28:
Interact with a household to solve a task. Imagine you are an intelligent agent in a household environment and your target is to perform actions to complete the task goal. 
At the beginning of your interactions, you will be given the detailed description of the current environment and your goal to accomplish. 

For each of your turn, you will be given a list of actions which you can choose one to perform in this turn.

You can choose either of given option: "THOUGHT" or "ACTION". If you choose "THOUGHT", you should first think about the current condition and plan for your future actions, 
and then output your action in this turn. 
Your output must strictly follow this format:"THOUGHT: your thoughts.
ACTION: your next action
"; 

If you choose "ACTION", you should directly output the action in this turn. Your output must strictly follow this format:"ACTION: your next action
".
 
 After your each turn, the environment will give you immediate feedback ba

In [48]:
print(env)

{'name': 'env_0', 'memory': [], 'is_success': False, 'skip': False}


In [69]:
update_memory(trial_log_path, env_configs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'name': 'env_0',
  'memory': ['You will be given the history of a past experience in which you were placed in an environment and given a task to complete. \n                      You were unsuccessful in completing the task. Do not summarize your environment, but rather think about the strategy and path \n                      you took to attempt to complete the task. Devise a concise, new plan of action that accounts for your mistake with reference \n                      to specific actions that you should have taken. For example, if you tried A and B but forgot C, \n                      then devise a plan to achieve C with environment-specific actions. You will need this later when you are solving the same task. \n                      Give your plan after "Plan". Here are two examples:\n\nYou are in the middle of a room. Looking quickly around you, you see a cabinet 10, a cabinet 9, a cabinet 8, a cabinet 7, a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabi

In [ ]:
json_file_path = "alfworld_conversations_20_29.json"
with open(json_file_path, "w", encoding="utf-8") as f:
    json.dump(conversations_traj, f, indent=2, ensure_ascii=False)

In [ ]:
generate_kwargs = {
    "do_sample": True,
    "temperature": 0.7,
    "max_new_tokens": 35,
    "max_length":30,
    "num_return_sequences":1
}
# actions = generator("Hello, I'm a language model", max_length = 30, num_return_sequences=3, temperature=0.1)
actions = generator("Hello, I'm a language model", **generate_kwargs)



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=35) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [ ]:
from typing import List, Optional
def get_llama_completion(model, tokenizer, prompt, temperature = 0.0, max_tokens = 256, stop_strs=['\n']) :
    # Tokenize the input
    print("prompt = ", prompt )
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            max_new_tokens=max_tokens,
            temperature=temperature,
            top_p=1.0,
            repetition_penalty=1.0,
            eos_token_id=tokenizer.eos_token_id,
            do_sample=True if temperature > 0 else False,
        )

    # Decode and extract response
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Apply stopping conditions manually (Hugging Face models do not support `stop` directly)
    if stop_strs:
        for stop in stop_strs:
            if stop in generated_text:
                generated_text = generated_text.split(stop)[0].strip()

    return generated_text


In [ ]:
def get_llama3_8b_response(model, tokenizer, messages):
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)
    # attention_mask = input_ids.ne(tokenizer.pad_token_id).to(DEVICE)
    if input_ids.shape[1] >= 4048:
        messages = messages[-10:]
        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            max_new_tokens=256,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
        )
    response = outputs[0][input_ids.shape[-1]:]
    result = tokenizer.decode(response, skip_special_tokens=True)
    # print(result)
    return result

In [ ]:
import gc
# To get the multiple response from the llm
def get_llama3_8b_response_multi(model, tokenizer, messages, k):
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    print(input_ids.shape)
    if input_ids.shape[1] >= 4048:
        messages = messages[-10:]
        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            max_new_tokens=256,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.9,
            top_p=0.95,
            num_return_sequences=k
        )
    # print(outputs.keys())
    # print(outputs['scores'])
    responses = []
    for i in range(k):
        response = outputs[i][input_ids.shape[-1]:]
        result = tokenizer.decode(response, skip_special_tokens=True)
        responses.append(result)
    # responses = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    # print(result)
    gc.collect()
    torch.cuda.empty_cache()
    return responses

In [ ]:
def cal_prob(target_text,messages,model,tokenizer):
    temp_messages = messages[:]
    temp_messages.append({'role':'assistant','content':target_text})
    input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
    encodings = tokenizer.apply_chat_template(temp_messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
    # print(input_ids.shape)
    # print(input_ids)
    # print(encodings.shape)
    # print(encodings)

    labels = encodings.clone()
    labels[:,:input_ids.shape[1]] = -100
    # print(labels)
    # print(labels)

    with torch.no_grad():
        outputs = model.forward(encodings, labels=labels)
        loss = outputs[0]
        text_prob=torch.exp(-loss)#**(len(target_text))
    return text_prob

In [ ]:
for a in actions:
  prob = cal_prob(a,messages,model,tokenizer)
  print(a,prob)

I don't have a personal name. I'm an artificial intelligence designed to assist and communicate with users in a helpful and informative way. You can think of me as a conversational AI or a chatbot. I'm here to provide information, answer questions, and engage in conversations to the best of my abilities. How can I assist you today? tensor(0.8483, device='cuda:0')
I'm an artificial intelligence model known as a conversational AI. I don't have a personal name, but you can think of me as a helpful assistant or a conversational partner. Some people call me "Assistant" or "AI," but feel free to give me a nickname if you'd like! tensor(0.8879, device='cuda:0')


In [ ]:
import os
import sys
import json
import yaml
import importlib
import alfworld

In [ ]:
from typing import List, Dict

class EnvironmentHistory:
    def __init__(self, base_query: str, start_info, memory: List[str], history: List[Dict[str, str]] = []) -> None:
        self._cur_query: str = f'{_get_base_query(base_query, start_info, memory)}'
        self._history: List[Dict[str, str]] = history
        self._last_action: str = ''
        self._is_exhausted: bool = False

    def add(self, label: str, value: str) -> None:
        assert label in ['action', 'observation', 'human_edit']
        self._history += [{
            'label': label,
            'value': value,
        }]
        if label == 'action':
            if value == self._last_action:
                self._is_exhausted = True
            else:
                self._last_action = value

    def check_is_exhausted(self) -> bool:
        return self._is_exhausted

    def reset(self) -> None:
        self._history = []

    def __str__(self) -> str:
        s: str = self._cur_query + '\n'
        for i, item in enumerate(self._history):
            if item['label'] == 'action':
                s += f'> {item["value"]}'
            elif item['label'] == 'observation':
                s += item['value']
            # NOT CURRENTLY SUPPORTED
            elif item['label'] == 'human_edit':
                s += f'[human edit]: {item["value"]}'
            if i != len(self._history) - 1:
                s += '\n'
        return s

def _get_base_query(base_query: str, start_info: str, memory: List[str]) -> str:
    query = base_query

    # add memory if it exists
    if len(memory) > 0:
        query += '\n\nYour memory for the task below:'
        for i, m in enumerate(memory):
            query += f'\nTrial {i}:\n{m.strip()}'
    query += f"\nHere is the task:\n{start_info}"
    return query


In [ ]:
from typing import Literal

Model = Literal["gpt-4", "gpt-3.5-turbo", "meta-llama/Llama-3.1-8B-Instruct"]

In [ ]:
with open("reflexion_few_shot_examples.txt", 'r') as f:
    FEW_SHOT_EXAMPLES = f.read()

In [ ]:
# from utils import get_completion

from typing import List, Dict, Any


def _get_scenario(s: str) -> str:
    """Parses the relevant scenario from the experience log."""
    return s.split("Here is the task:")[-1].strip()

def _generate_reflection_query(log_str: str, memory: List[str]) -> str:
    """Allows the Agent to reflect upon a past experience."""
    scenario: str = _get_scenario(log_str)
    query: str = f"""You will be given the history of a past experience in which you were placed in an environment and given a task to complete.
                      You were unsuccessful in completing the task. Do not summarize your environment, but rather think about the strategy and path
                      you took to attempt to complete the task. Devise a concise, new plan of action that accounts for your mistake with reference
                      to specific actions that you should have taken. For example, if you tried A and B but forgot C,
                      then devise a plan to achieve C with environment-specific actions. You will need this later when you are solving the same task.
                      Give your plan after "Plan". Here are two examples:

{FEW_SHOT_EXAMPLES}

{scenario}"""

    if len(memory) > 0:
        query += '\n\nPlans from past attempts:\n'
        for i, m in enumerate(memory):
            query += f'Trial #{i}: {m}\n'

    query += '\n\nNew plan:'
    return query

def update_memory(trial_log_path: str, env_configs: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """Updates the given env_config with the appropriate reflections."""
    with open(trial_log_path, 'r') as f:
        full_log: str = f.read()

    env_logs: List[str] = full_log.split('#####\n\n#####')
    assert len(env_logs) == len(env_configs), print(f'bad: {len(env_logs)}, {len(env_configs)}')
    for i, env in enumerate(env_configs):
        # if unsolved, get reflection and update env config
        if not env['is_success'] and not env['skip']:
            if len(env['memory']) > 3:
                memory: List[str] = env['memory'][-3:]
            else:
                memory: List[str] = env['memory']
            reflection_query: str = _generate_reflection_query(env_logs[i], memory)
            reflection: str = get_llama3_8b_response(reflection_query) # type: ignore
            env_configs[i]['memory'] += [reflection]

    return env_configs


In [ ]:
from typing import List, Dict, Any, Tuple

def process_ob(ob):
    if ob.startswith('You arrive at loc '):
        ob = ob[ob.find('. ')+2:]
    return ob

def alfworld_run(env, base_prompt, memory: List[str], to_print=True, ob='', model: Model = "text-davinci-003") -> Tuple[EnvironmentHistory, bool]:
    if len(memory) > 3:
        env_history = EnvironmentHistory(base_prompt, ob, memory[-3:], [])
    else:
        env_history = EnvironmentHistory(base_prompt, ob, memory, [])
    env_history.reset()
    if to_print:
        print(ob)
        sys.stdout.flush()
    cur_step = 0
    while cur_step < 15:
        # action = generator(model=model, tokenizer= tokenizer, messages = str(env_history) + ">", k=1).strip()
        action = generator(str(env_history)+">", num_return_sequences=1)
        action = action[0]['generated_text'].strip()

        # print(f'action1 = {action1}')
        # print(f'type(action1) = {type(action1)}')

        env_history.add("action", action)
        observation, reward, done, info = env.step([action])
        observation, reward, done = process_ob(observation[0]), info['won'][0], done[0]
        if action.startswith('think:'):
            observation = 'OK.'
        env_history.add("observation", observation)
        if to_print:
            print(f'> {action}\n{observation}')
            sys.stdout.flush()
        if done:
            return env_history, True
        elif env_history.check_is_exhausted():
            return env_history, False
        cur_step += 1
    return env_history, False

In [ ]:
alfworld_prompt = """Interact with a household environment to complete a given task.
You are an intelligent agent, and your goal is to plan and execute actions efficiently to complete the task goal.

### Instructions:
- At the beginning, you will receive a **detailed description of current environment** and a **specific task** to accomplish.
- In **each turn**, you must select **one action** from the available list.
- You can either **think (THOUGHT)** or **act (ACTION)**.

### Action Rules:
1. You **must strictly follow one of these formats**:
   - **THOUGHT format** (for reasoning before an action):
     ```
     THOUGHT: [Your thoughts]
     ACTION: [Your next action]
     ```
   - **ACTION format** (if reasoning is not needed):
     ```
     ACTION: [Your next action]
     ```

### Interaction Process:
- After each action, you will receive **immediate feedback** from the environment.
- If the environment outputs `"Nothing happened"`, it means the **previous action was invalid**, and you must try another option.

### Important Reminders:
**Only choose actions from the provided "available actions" list.**
**Any action outside this list is considered illegal.**
**Think when necessary, but act directly when confident.**

"""

In [ ]:
PREFIXES = {
    'pick_and_place': 'put',
    'pick_clean_then_place': 'clean',
    'pick_heat_then_place': 'heat',
    'pick_cool_then_place': 'cool',
    'look_at_obj': 'examine',
    'pick_two_obj': 'puttwo'
}

def run_trial(
        trial_log_path: str,
        world_log_path: str,
        trial_idx: int,
        env_configs: List[Dict[str, Any]],
        use_memory: bool,
        model: Model,
    ) -> List[Dict[str, Any]]:
    importlib.reload(alfworld)
    importlib.reload(alfworld.agents.environment.alfred_tw_env)

    with open('base_config.yaml') as reader:
        config = yaml.safe_load(reader)
    split = "eval_out_of_distribution"

    # env = getattr(alfworld.agents.environment, config["env"]["type"])(config, train_eval=split)
    env = getattr(alfworld.agents.environment.alfred_tw_env, 'AlfredTWEnv')(config, train_eval=split)

    env = env.init_env(batch_size=1)

    num_successes: int = 0
    num_additional_successes: int = 0
    num_envs: int = len(env_configs)

    for z, env_config in enumerate(env_configs):
        ob, info = env.reset()
        ob = '\n'.join(ob[0].split('\n\n')[1:])
        name = '/'.join(info['extra.gamefile'][0].split('/')[-3:-1])

        print(f"using {name}")

        if env_config["is_success"]:
            num_successes += 1

            # log to world log
            with open(world_log_path, 'a') as wf:
                wf.write(f'Environment #{z} Trial #{trial_idx}: SUCCESS\n')
            with open(trial_log_path, 'a') as wf:
                wf.write(f'\n#####\n\nEnvironment #{z}: Success\n\n#####\n')
            continue



        for i, (k, v) in enumerate(PREFIXES.items()):
            if name.startswith(k):
                base_prompt = 'Interact with a household to solve a task. Here are two examples.\n' + d[f'react_{v}_1'] + d[f'react_{v}_0']

                final_env_history, is_success = alfworld_run(env, base_prompt, env_config["memory"] if use_memory else [], to_print=True, ob=ob, model=model)

                # update env config
                if is_success:
                    status_str: str = f'Environment #{z} Trial #{trial_idx}: SUCCESS'
                    env_configs[z]['is_success'] = True
                    num_successes += 1
                    num_additional_successes += 1
                else:
                    status_str: str = f'Environment #{z} Trial #{trial_idx}: FAIL'

                # log to world log
                with open(world_log_path, 'a') as f:
                    f.write(status_str + '\n')

                # log env results to trial log
                with open(trial_log_path, 'a') as wf:
                    wf.write(f'\n#####\n\nEnvironment #{z}:\n{str(final_env_history)}\n\nSTATUS: {"OK" if is_success else "FAIL"}\n\n#####\n')

    # close environment object
    env.close()

    # log trial results to trial and world logs
    log_str: str = f"""
-----
SUCCESS: {num_successes}
ADDITIONAL SUCCESS: {num_additional_successes}
FAIL: {num_envs - num_successes}
TOTAL: {num_envs}
ACCURACY: {round(num_successes / num_envs, 2)}
-----"""
    with open(trial_log_path, 'a') as wf:
        wf.write(log_str)
    with open(world_log_path, 'a') as wf:
        wf.write(log_str + '\n')

    return env_configs


In [ ]:
PROMPT_FILE = 'alfworld_3prompts.json'
with open(PROMPT_FILE, 'r') as f:
    d = json.load(f)

In [ ]:
num_envs = 2
num_trials = 3
trial_log_path = 'trial_log.log'
world_log_path = 'world.log'
trial_idx = 0
env_configs: List[Dict[str, Any]] = []
for i in range(num_envs):
    env_configs += [{
        'name': f'env_{i}',
        'memory': [],
        'is_success': False,
        'skip': False
    }]



In [ ]:
use_memory = True
run_trial(trial_log_path, world_log_path, trial_idx, env_configs, use_memory, model)

Initializing AlfredTWEnv...


100%|██████████| 341/341 [00:00<00:00, 583.85it/s]


Overall we have 134 games in split=eval_out_of_distribution
Evaluating with 134 games
using pick_cool_then_place_in_recep-Tomato-None-Microwave-10/trial_T20190909_102644_926781
You are in the middle of a room. Looking quickly around you, you see a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 3, a countertop 2, a countertop 1, a drawer 3, a drawer 2, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a shelf 3, a shelf 2, a shelf 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1.
Your task is to: put a cool tomato in microwave.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


> Interact with a household to solve a task. Here are two examples.
You are in the middle of a room. Looking quickly around you, you see a cabinet 9, a cabinet 8, a cabinet 7, a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 3, a countertop 2, a countertop 1, a drawer 9, a drawer 8, a drawer 7, a drawer 6, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a shelf 3, a shelf 2, a shelf 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1.
Your task is to: put a cool mug in shelf.
> think: To solve the task, I need to find and take a mug, then cool it with fridge, then put it in shelf.
OK.
> think: First I need to find a mug. A mug is more likely to appear in countertop (1-3), coffeemachine (1), cabinet (1-9), shelf (1-3), drawer (1-9). I can check one by one, starting with countertop 1.
OK.
> go to countertop 1
On the countert

...

> Interact with a household to solve a task. Here are two examples.
You are in the middle of a room. Looking quickly around you, you see a cabinet 9, a cabinet 8, a cabinet 7, a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 3, a countertop 2, a countertop 1, a drawer 9, a drawer 8, a drawer 7, a drawer 6, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a shelf 3, a shelf 2, a shelf 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1.
Your task is to: put a cool mug in shelf.
> think: To solve the task, I need to find and take a mug, then cool it with fridge, then put it in shelf.
OK.
> think: First I need to find a mug. A mug is more likely to appear in countertop (1-3), coffeemachine (1), cabinet (1-9), shelf (1-3), drawer (1-9). I can check one by one, starting with countertop 1.
OK.
> go to countertop 1
On the countert

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


> Interact with a household to solve a task. Here are two examples.
You are in the middle of a room. Looking quickly around you, you see a cabinet 9, a cabinet 8, a cabinet 7, a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 3, a countertop 2, a countertop 1, a drawer 9, a drawer 8, a drawer 7, a drawer 6, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a shelf 3, a shelf 2, a shelf 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1.
Your task is to: put a cool mug in shelf.
> think: To solve the task, I need to find and take a mug, then cool it with fridge, then put it in shelf.
OK.
> think: First I need to find a mug. A mug is more likely to appear in countertop (1-3), coffeemachine (1), cabinet (1-9), shelf (1-3), drawer (1-9). I can check one by one, starting with countertop 1.
OK.
> go to countertop 1
On the countert

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.50 GiB. GPU 0 has a total capacity of 14.74 GiB of which 984.12 MiB is free. Process 12303 has 13.78 GiB memory in use. Of the allocated memory 10.50 GiB is allocated by PyTorch, and 3.15 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)